# Observability Tutorial

This notebook explores **observability** from multiple perspectives:

- Control-theoretic observability matrix (linear case)
- Observability Gramian (controllability dual)
- Nullspace-based tests for individual state observability
- Statistical viewpoint: Cramér–Rao lower bound (CRLB)
- Nonlinear systems and partial observability

It builds on the theory described in the `pykal` documentation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import matrix_rank, svd, eigvals, pinv
from scipy.linalg import expm, solve_continuous_are
from sympy import symbols, Matrix, simplify, sin, cos, lambdify, eye
from scipy.integrate import solve_ivp


## 1. Control-Theoretic Observability: Linear Systems
We start with a basic example: a 2D position–velocity system.

In [ ]:
A = np.array([[0, 1],
              [0, 0]])
H = np.array([[1, 0]])

n = A.shape[0]
O = np.vstack([H @ np.linalg.matrix_power(A, i) for i in range(n)])
rank_O = matrix_rank(O)

print("Observability matrix O:")
print(O)
print(f"Rank of O = {rank_O} (out of {n}) => {'Observable' if rank_O == n else 'Not Observable'}")


## 2. Observability Gramian (Continuous-Time)
The observability Gramian is the solution to:

\[ W_o = \int_0^T e^{A^\top t} H^\top H e^{At} dt \]

In [ ]:
T = 5.0
def observability_gramian(A, H, T, dt=0.01):
    G = np.zeros((A.shape[0], A.shape[0]))
    for t in np.arange(0, T, dt):
        eAt = expm(A * t)
        G += (eAt.T @ H.T @ H @ eAt) * dt
    return G

G = observability_gramian(A, H, T)
print("Observability Gramian:")
print(G)
print(f"Rank = {np.linalg.matrix_rank(G)}")


## 3. Individual State Observability (Nullspace Test)
We compute the nullspace of the observability matrix to identify unobservable directions.

In [ ]:
from scipy.linalg import null_space

N = null_space(O)
print("Nullspace of O (unobservable directions):")
print(N)

if N.shape[1] > 0:
    print("Some directions are unobservable.")
else:
    print("All directions are observable.")


## 4. Statistical Observability: CRLB
Given a linear system:

\[ y = Hx + v, \quad v \sim \mathcal{N}(0, R) \]

The CRLB for unbiased estimators is:

\[ \text{Cov}(\hat{x}) \succeq (H^\top R^{-1} H)^{-1} \]

In [ ]:
R = 0.01 * np.eye(H.shape[0])
FIM = H.T @ np.linalg.inv(R) @ H

try:
    CRLB = np.linalg.inv(FIM)
    print("CRLB exists. State is statistically observable.")
except np.linalg.LinAlgError:
    print("CRLB does not exist (FIM singular). State not observable.")


## 5. Nonlinear Observability (EKF-style Linearization)
Consider the damped pendulum with unknown state \( x = [\theta, \dot{\theta}] \). We measure only angle \( \theta \).

## Conclusion
This notebook demonstrated how to:
- Analyze nonlinear systems (Van der Pol)
- Construct observability matrices and Gramians
- Detect partial observability via nullspaces
- Estimate fundamental limits on estimation using CRLB

Observability is not binary: it can be weak, partial, or local.